Auteur: Michaël Leroy

Requetes OMS des POI   ( Deprecated) 

[+] API: 
    
    - Nominatim: https://nominatim.openstreetmap.org/
        * obtention des surfaces en shapely geometries
    - OpenStreetMaps :
        * Lib OSMnx, obtention des résaux routiers sous forme de graph 
    - OpenChargeMap : 
        * Points de charge EV (data.gouv et autres)        

[+] Méthodologie

    - Récupération des réseaux routiers:
        * Nodes: position des intersections routiéres
        * Edges: shapely LineString, distance, limitation de vitesse, types de voies...
    - Récupération des bornes EV
        * Création d'un noeud sur l'edge le plus proche
        * Split Linestring
        # Branchement du noeud et raccord aus deux noeud 'parents'

[+] Utilité

    - Graph Stats:
        betweeness, closeness.....

[+] A ajouter pour stats:

    - On peut aussi avoir des noeuds 'amenity', ie hotels, parkings, parcs attraction...               

[-] Au dela de 3 Régions explosion de la mémoire lors de la construction du graph.


https://wiki.openstreetmap.org/wiki/Key:highway

In [1]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

['CFG.pkl']

In [2]:
CFG.area_name

'france'

In [3]:
import os
os.environ['USE_PYGEOS'] = '0'

In [4]:
%load_ext autoreload
%autoreload



In [5]:

%aimport OSM_helper_functions
%aimport OCM_helper_functions


In [6]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

' To retrieve http data from OSMnx and cache them locally\n    before building graph. Raise and exception if requests\n    are all done. intercept then set to False and retry.'

In [7]:
import pdb
import functools

def debugger_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            pdb.post_mortem()
            raise e
    return wrapper

# Listes des découpages Régions Départements

In [8]:
Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    'Corse-du-Sud': '2A',
    'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


In [9]:
# selected_regions = [
#     # 'Larzicourt',
#     # 'Arrigny'
#     # 'Reims'
#     # 'Lot',
#     # 'Aube',
#     'Ardennes',
#     # 'Marne',
#     # 'Haute-Marne',
#     # 'Côtes-d\'Armor',
#     # 'Bretagne',
#     # 'Île-de-France'
#     # 'Grand Est', 
#     # 'Hauts-de-France',
#     # 'Bourgogne-Franche-Comté',
#     # 'Auvergne-Rhône-Alpes',
#     # 'Provence-Alpes-Côte d\'Azur'

# ]
# # selected_regions = CFG.area

In [10]:
try:
    selection = []
    for reg in CFG.area:
        selection += Administratives[reg]
except:
    print('Not a Région')
    selection= CFG.area
selection

['Ain',
 'Allier',
 'Ardèche',
 'Cantal',
 'Drôme',
 'Isère',
 'Loire',
 'Haute-Loire',
 'Puy-de-Dôme',
 'Rhône',
 'Savoie',
 'Haute-Savoie',
 "Côte-d'Or",
 'Doubs',
 'Jura',
 'Nièvre',
 'Haute-Saône',
 'Saône-et-Loire',
 'Yonne',
 'Territoire de Belfort',
 "Côtes-d'Armor",
 'Finistère',
 'Ille-et-Vilaine',
 'Morbihan',
 'Cher',
 'Eure-et-Loir',
 'Indre',
 'Indre-et-Loire',
 'Loir-et-Cher',
 'Loiret',
 'Ardennes',
 'Aube',
 'Marne',
 'Haute-Marne',
 'Meurthe-et-Moselle',
 'Meuse',
 'Moselle',
 'Bas-Rhin',
 'Haut-Rhin',
 'Vosges',
 'Aisne',
 'Nord',
 'Oise',
 'Pas-de-Calais',
 'Somme',
 'Paris',
 'Seine-et-Marne',
 'Yvelines',
 'Essonne',
 'Hauts-de-Seine',
 'Seine-Saint-Denis',
 'Val-de-Marne',
 "Val-d'Oise",
 'Calvados',
 'Eure',
 'Manche',
 'Orne',
 'Seine-Maritime',
 'Charente',
 'Charente-Maritime',
 'Corrèze',
 'Creuse',
 'Dordogne',
 'Gironde',
 'Landes',
 'Lot-et-Garonne',
 'Pyrénées-Atlantiques',
 'Deux-Sèvres',
 'Vienne',
 'Haute-Vienne',
 'Ariège',
 'Aude',
 'Aveyron',
 'Gard

# Graph construction

## Get Graph

In [11]:
CFG.roads

'|motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link'

In [12]:
AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

['Ain',
 'Allier',
 'Ardèche',
 'Cantal',
 'Drôme',
 'Isère',
 'Loire',
 'Haute-Loire',
 'Puy-de-Dôme',
 'Rhône',
 'Savoie',
 'Haute-Savoie',
 "Côte-d'Or",
 'Doubs',
 'Jura',
 'Nièvre',
 'Haute-Saône',
 'Saône-et-Loire',
 'Yonne',
 'Territoire de Belfort',
 "Côtes-d'Armor",
 'Finistère',
 'Ille-et-Vilaine',
 'Morbihan',
 'Cher',
 'Eure-et-Loir',
 'Indre',
 'Indre-et-Loire',
 'Loir-et-Cher',
 'Loiret',
 'Ardennes',
 'Aube',
 'Marne',
 'Haute-Marne',
 'Meurthe-et-Moselle',
 'Meuse',
 'Moselle',
 'Bas-Rhin',
 'Haut-Rhin',
 'Vosges',
 'Aisne',
 'Nord',
 'Oise',
 'Pas-de-Calais',
 'Somme',
 'Paris',
 'Seine-et-Marne',
 'Yvelines',
 'Essonne',
 'Hauts-de-Seine',
 'Seine-Saint-Denis',
 'Val-de-Marne',
 "Val-d'Oise",
 'Calvados',
 'Eure',
 'Manche',
 'Orne',
 'Seine-Maritime',
 'Charente',
 'Charente-Maritime',
 'Corrèze',
 'Creuse',
 'Dordogne',
 'Gironde',
 'Landes',
 'Lot-et-Garonne',
 'Pyrénées-Atlantiques',
 'Deux-Sèvres',
 'Vienne',
 'Haute-Vienne',
 'Ariège',
 'Aude',
 'Aveyron',
 'Gard

In [13]:
# By graphs
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]graph.graphml')
try:
    print(f'Datas retrieved for {CFG.area_name}', end='  ')
    G = ox.io.load_graphml(file)
    print(f'{G.number_of_nodes()} nodes, {G.number_of_edges()} edges')

except:
    print('Datas not found')
    G = nx.MultiDiGraph()
    stats = {}
    total = len(AREA)
    print(CFG.roads)
    for n, area in enumerate(AREA):
        print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
        G_ = get_graph(area, custom_filter=CFG.roads)
        G_ = fill_missing_edge_geometry(G_)
        G = nx.compose(G, G_)
        stats[area] = (G_.number_of_nodes(), G_.number_of_edges()   )
        del G_
    display(stats)
    print(f'before:          {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.')
    G =  ox.utils_graph.get_largest_component(G, strongly=True)
    print(f'keep largest:    {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.')
    G =  ox.utils_graph.remove_isolated_nodes(G)
    print(f'remove isolated: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.')

    ox.io.save_graphml(G, file) 

nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
# G.number_of_nodes(), G.number_of_edges()        


Datas retrieved for france  Datas not found
|motorway|motorway_link|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link
Datas retrieved for Ain: 1/94 ~ 1.06%
All requests done, building graph... add speed, distances, travel time, simplify... done. 
> 5334 nodes and 11001 edges,  722 edges geometries corrected.
Datas retrieved for Allier: 2/94 ~ 2.13%
All requests done, building graph... add speed, distances, travel time, simplify... done. 
> 4742 nodes and 10251 edges,  932 edges geometries corrected.
Datas retrieved for Ardèche: 3/94 ~ 3.19%
All requests done, building graph... add speed, distances, travel time, simplify... done. 
> 3583 nodes and 7048 edges,  609 edges geometries corrected.
Datas retrieved for Cantal: 4/94 ~ 4.26%
All requests done, building graph... add speed, distances, travel time, simplify... done. 
> 2280 nodes and 5340 edges,  329 edges geometries corrected.
Datas retrieved for Drôme: 5/94 ~ 5.32%
All requests done, building gr

{'Ain': (5334, 11001),
 'Allier': (4742, 10251),
 'Ardèche': (3583, 7048),
 'Cantal': (2280, 5340),
 'Drôme': (7244, 13717),
 'Isère': (9201, 17799),
 'Loire': (5421, 10814),
 'Haute-Loire': (2825, 6029),
 'Puy-de-Dôme': (8072, 17008),
 'Rhône': (4941, 9565),
 'Savoie': (5363, 10174),
 'Haute-Savoie': (7797, 14435),
 "Côte-d'Or": (5550, 12216),
 'Doubs': (4288, 8797),
 'Jura': (3598, 7965),
 'Nièvre': (3199, 7182),
 'Haute-Saône': (3325, 7068),
 'Saône-et-Loire': (5032, 11037),
 'Yonne': (3672, 8615),
 'Territoire de Belfort': (1188, 2320),
 "Côtes-d'Armor": (7290, 14885),
 'Finistère': (10937, 21152),
 'Ille-et-Vilaine': (10783, 21422),
 'Morbihan': (9345, 17918),
 'Cher': (3629, 8151),
 'Eure-et-Loir': (7401, 17664),
 'Indre': (3148, 7481),
 'Indre-et-Loire': (5510, 10912),
 'Loir-et-Cher': (3418, 7178),
 'Loiret': (5830, 11708),
 'Ardennes': (2743, 6149),
 'Aube': (3826, 8988),
 'Marne': (5315, 11079),
 'Haute-Marne': (3028, 7032),
 'Meurthe-et-Moselle': (5433, 10970),
 'Meuse': (24

before:          537732 nodes, 1089656 edges.
keep largest:    529853 nodes, 1080531 edges.
remove isolated: 529853 nodes, 1080531 edges.


In [14]:
nodes.shape, edges.shape

((529853, 6), (1080531, 20))

In [15]:
del G, nodes, edges

In [16]:
def missing(df):
    return pd.DataFrame({'column_name': df.columns,
                         'percent_missing': df.isnull().sum() * 100 / len(df)
                         }).sort_values(by='percent_missing',ascending=True)

In [38]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'amenity': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
amenity,amenity,0.000000
geometry,geometry,0.000000
nodes,nodes,36.966165
name,name,79.043738
source,source,83.881783
...,...,...
pets_allowed,pets_allowed,99.994842
managed,managed,99.994842
theme,theme,99.994842
recycling:engine_oil,recycling:engine_oil,99.994842


In [47]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]amenity_tags.feather')
try:
  tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      tags = pd.concat(
        [tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'amenity': True}, 
                                        which_result=None, buffer_dist=None)[['amenity','geometry']]
        ]
      )
    tags.to_feather(file)

display(tags.info())
display(tags['amenity'].value_counts())

del tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

parking            375566
parking_space      262286
bench              174167
waste_basket        89289
bicycle_parking     83931
                    ...  
orthophonist            1
playroom                1
bread_machine           1
dancing school          1
B4                      1
Name: amenity, Length: 863, dtype: int64

In [17]:
# file = os.path.join(CFG.data_path, f'[{CFG.area_name}]social_tags.feather')
# try:
#   social_tags = gpd.read_feather(file)
# except:
#     print('Datas not found on disk')
#     social_tags = gpd.GeoDataFrame()
#     total = len(AREA)
#     for n, area in enumerate(AREA):
#       print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
#       social_tags = pd.concat(
#         [social_tags,
#          ox.geometries.geometries_from_place(area + ', Metropolitan France',
#                                         tags={'amenity':['food_court',
#                                                           'restaurant',
#                                                           'arts_centre',
#                                                           'cinema',
#                                                           'community_centre',
#                                                           'conference_centre',
#                                                           'events_venue',
#                                                           'exhibition_center',
#                                                           'music_venue',
#                                                           'planetarium',
#                                                           'theatre'],
#                                               }, 
#                                         which_result=None, buffer_dist=None)[['amenity','geometry',]]
#         ]
#       )
#     # social_tags = social_tags[['amenity','geometry',]]
#     social_tags.to_feather(file)

# display(social_tags.info())
# display(social_tags['amenity'].value_counts())

# del social_tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

restaurant           75587
community_centre     16772
theatre               2843
arts_centre           2456
cinema                2011
events_venue           325
food_court             190
conference_centre      126
planetarium             28
music_venue             24
Name: amenity, dtype: int64

# Historic

In [39]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'historic':True }, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
geometry,geometry,0.000000
historic,historic,0.000000
name,name,42.517344
nodes,nodes,68.384539
source,source,72.745292
...,...,...
ref:FR:LaPoste,ref:FR:LaPoste,99.900892
stamping_machine,stamping_machine,99.900892
government,government,99.900892
basilica,basilica,99.900892


In [19]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]historic_tags.feather')
try:
  historic_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    historic_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      historic_tags = pd.concat(
        [historic_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'historic':True }, 
                                     which_result=None, buffer_dist=None)[['historic','geometry']]
        ]
      )
    # historic_tags = historic_tags[['historic','amenity','geometry','source']]
    historic_tags.to_feather(file)                                   

display(historic_tags.info())
display(historic_tags['historic'].value_counts())

del historic_tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

wayside_cross          41185
memorial               28462
castle                 14516
ruins                   9109
archaeological_site     8724
                       ...  
tunnel                     1
apartments                 1
trench                     1
crater                     1
castle;ruins               1
Name: historic, Length: 361, dtype: int64

# Natural

In [45]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'natural': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
geometry,geometry,0.000000
natural,natural,0.000000
nodes,nodes,72.654839
source,source,73.419187
leaf_type,leaf_type,80.942234
...,...,...
access,access,99.995368
source:date,source:date,99.995368
leisure,leisure,99.995368
emergency,emergency,99.995368


In [46]:

file = os.path.join(CFG.data_path, f'[{CFG.area_name}]natural_tags.feather')
try:
  natural_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    natural_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      natural_tags = pd.concat(
        [natural_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'natural':True }, 
                                     which_result=None, buffer_dist=None)[['natural','geometry']]
        ]
      )
    natural_tags.to_feather(file)                                   

display(natural_tags.info())
display(natural_tags['natural'].value_counts())

del natural_tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

tree            1769071
water            499041
wood             256264
tree_row         182404
scrub            148346
                 ...   
locality              1
volcano;peak          1
crater                1
water;wood            1
proposed              1
Name: natural, Length: 137, dtype: int64

# Charge Stations

In [40]:
# test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
#                                         tags={'amenity':['charging_station']}, 
#                                      which_result=None, buffer_dist=None)
# missing(test)

,column_name,percent_missing
amenity,amenity,0.000000
geometry,geometry,0.000000
capacity,capacity,36.170213
fee,fee,40.425532
name,name,44.680851
...,...,...
socket:schuko:output,socket:schuko:output,98.936170
payment:credit_cards,payment:credit_cards,98.936170
ogr_type_c,ogr_type_c,98.936170
operator_1,operator_1,98.936170


In [25]:
# file = os.path.join(CFG.data_path, f'[{CFG.area_name}]charge_tags.feather')
# try:
#   charge_tags = gpd.read_feather(file)
# except:
#     print('Datas not found on disk')
#     charge_tags = gpd.GeoDataFrame()
#     total = len(AREA)
#     for n, area in enumerate(AREA):
#       print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
#       charge_tags = pd.concat(
#         [charge_tags,
#          ox.geometries.geometries_from_place(area + ', Metropolitan France',
#                                         tags={'amenity':['charging_station']}, 
#                                      which_result=None, buffer_dist=None)
#                                     #  [[
#                                     # 'amenity', 
#                               #  'capacity',
#                               #  'geometry',
#       #                          'socket:type2',
#       #                          'socket:typee',
#       #                          'socket:type3', 'source','socket:tesla_supercharger',
#       #  'socket:tesla_supercharger:output','socket:tesla_supercharger_ccs:output',
#       #  'socket:type2_combo', 'socket:type2_combo:output','socket:chademo',
#       #  'socket:type3c',
#       # 'amperage',
#       #  'voltage','socket:schuko','source:socket:type2:output', 'source:socket:typee',
#       #  'capacity:charging', 'charging_station:output','socket:chademo:current',
#       #  'socket:type2:current', 'socket:type2_combo:current'
#                       # ]]
#         ]
#       )
#     charge_tags = charge_tags[['amenity', 
#                                'capacity',
#                                'geometry',
#                                'socket:type2',
#                                'socket:typee',
#                                'socket:type3', 'source','socket:tesla_supercharger',
#        'socket:tesla_supercharger:output','socket:tesla_supercharger_ccs:output',
#        'socket:type2_combo', 'socket:type2_combo:output','socket:chademo',
#        'socket:type3c','amperage',
#        'voltage','socket:schuko','source:socket:type2:output', 'source:socket:typee',
#        'capacity:charging', 'charging_station:output','socket:chademo:current',
#        'socket:type2:current', 'socket:type2_combo:current'
#                       ]]
#     charge_tags.to_feather(file)                                                

# display(charge_tags.info())
# display(charge_tags['amenity'].value_counts())

# del charge_tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

charging_station    10255
Name: amenity, dtype: int64

In [41]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'leisure':True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
leisure,leisure,0.000000
geometry,geometry,0.000000
nodes,nodes,7.903001
access,access,59.874389
sport,sport,74.162596
...,...,...
CLC:year,CLC:year,99.982554
leaf_type,leaf_type,99.982554
mhs:inscription_date_1,mhs:inscription_date_1,99.982554
departures_board,departures_board,99.982554


In [26]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]leisure_tags.feather')
try:
  leisure_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    leisure_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      leisure_tags = pd.concat(
        [leisure_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'leisure':True}, 
                                     which_result=None, buffer_dist=None)
                                     [
                                                                          [
                                                                          'leisure',
                                                                          'geometry',
                                                                          ]
                                                                           ]
        ]
      )
    leisure_tags.to_feather(file)                                    

display(leisure_tags.info())
display(leisure_tags['leisure'].value_counts())

del leisure_tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

swimming_pool                                                           578282
pitch                                                                   149983
garden                                                                   47455
picnic_table                                                             34756
playground                                                               32873
                                                                         ...  
picnic_tableamenity=shelter_building=yes_shelter_type=picnic_shelter         1
wimming_area                                                                 1
sports_centre;sports_hall                                                    1
sauna;hot_tub                                                                1
sports_centre; amusement_arcade                                              1
Name: leisure, Length: 233, dtype: int64

In [42]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'sport':True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
sport,sport,0.0
geometry,geometry,0.0
leisure,leisure,12.5
nodes,nodes,32.5
name,name,37.5
source,source,70.0
addr:housenumber,addr:housenumber,77.5
landuse,landuse,85.0
operator:type,operator:type,87.5
operator,operator,87.5


In [49]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]sport_tags.feather')
try:
  sport_tags = gpd.read_feather(file)
except:
    print('Datas not found on disk')
    sport_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      sport_tags = pd.concat(
        [sport_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'sport':True}, 
                                     which_result=None, buffer_dist=None)
                                     [
                                                                      ['sport', 
                                                                       'geometry']
                                                                        ]
        ]                                                               
      )
    
    sport_tags.to_feather(file)
                                  

display(sport_tags.info())
display(sport_tags['sport'].value_counts())

del sport_tags

Datas not found on disk
Datas retrieved for Ain: 1/94 ~ 1.06%
Datas retrieved for Allier: 2/94 ~ 2.13%
Datas retrieved for Ardèche: 3/94 ~ 3.19%
Datas retrieved for Cantal: 4/94 ~ 4.26%
Datas retrieved for Drôme: 5/94 ~ 5.32%
Datas retrieved for Isère: 6/94 ~ 6.38%
Datas retrieved for Loire: 7/94 ~ 7.45%
Datas retrieved for Haute-Loire: 8/94 ~ 8.51%
Datas retrieved for Puy-de-Dôme: 9/94 ~ 9.57%
Datas retrieved for Rhône: 10/94 ~ 10.64%
Datas retrieved for Savoie: 11/94 ~ 11.70%
Datas retrieved for Haute-Savoie: 12/94 ~ 12.77%
Datas retrieved for Côte-d'Or: 13/94 ~ 13.83%
Datas retrieved for Doubs: 14/94 ~ 14.89%
Datas retrieved for Jura: 15/94 ~ 15.96%
Datas retrieved for Nièvre: 16/94 ~ 17.02%
Datas retrieved for Haute-Saône: 17/94 ~ 18.09%
Datas retrieved for Saône-et-Loire: 18/94 ~ 19.15%
Datas retrieved for Yonne: 19/94 ~ 20.21%
Datas retrieved for Territoire de Belfort: 20/94 ~ 21.28%
Datas retrieved for Côtes-d'Armor: 21/94 ~ 22.34%
Datas retrieved for Finistère: 22/94 ~ 23.40%
D

None

tennis                                                                     47464
soccer                                                                     35695
multi                                                                      17194
boules                                                                     13460
basketball                                                                  8505
                                                                           ...  
soccer;tennis;multi;running;athletics;fitness;table_tennis;archery;judo        1
fitness;exercise;musculation;cours_collectifs                                  1
basket;handball                                                                1
skateboard;roller_skating;bmx;basketball;football;handball                     1
volleyball;handball;football;basketball                                        1
Name: sport, Length: 2172, dtype: int64

In [43]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'boundary':'administrative'}, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
geometry,geometry,0.000000
admin_level,admin_level,0.000000
nodes,nodes,0.000000
boundary,boundary,0.000000
source,source,27.662296
...,...,...
name:syc,name:syc,99.968020
operator,operator,99.968020
note:source,note:source,99.968020
contact:website,contact:website,99.968020


In [ ]:
# TODO: rename file with []
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]boundaries.feather')
try:
  boundaries = gpd.read_feather(file)
except:
    print('Datas not found')
    boundaries = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      boundaries = pd.concat(
        [boundaries,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'boundary':'administrative'}, 
                                     which_result=1, buffer_dist=None)
                    #                  [['name', 
                    #            'geometry',
                    #   ]]
        ]
      )
    boundaries = boundaries[[
      'boundary','geometry','admin_level'
    ]]
    boundaries.to_feather(file)                                  

display(boundaries.info())
# tourism_tags['amenity'].value_counts()   
boundaries                                 

In [44]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'tourism': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

,column_name,percent_missing
tourism,tourism,0.000000
geometry,geometry,0.000000
name,name,36.610487
information,information,74.812734
nodes,nodes,76.591760
...,...,...
building:architecture,building:architecture,99.906367
site_type,site_type,99.906367
location,location,99.906367
architect,architect,99.906367


In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]tourism_tags.feather')
try:
  tourism_tags = gpd.read_feather(file)
except:
    print('Datas not found')
    tourism_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      tourism_tags = pd.concat(
        [tourism_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'tourism': True}, 
                                     which_result=None, buffer_dist=None)
                                     [['tourism', 
                               'geometry',
                      ]]
        ]
      )
   
    tourism_tags.to_feather(file)                                  

display(tourism_tags.info())
display(tourism_tags['tourism'].value_counts())

del tourism_tags

In [ ]:
# TODO: get each commune Mairie coordinates

In [ ]:
# TODO: merge decoupage with osm_id and Mairies coordinates

In [ ]:
*    End of OSM datasets file retrieval

In [ ]:
# Add isom_id to communes decoupage, for graph building

import requests, io
data_path = './datas'
file_name = 'decoupage_administratif'
ext = '.json'
url ='https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581'


# import requests
# from requests.adapters import HTTPAdapter
# import urllib3
# from urllib3 import Retry

# session = requests.Session()
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
# session.headers.update(headers)
# adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
# session.mount("http://", adapter)
# session.mount("https://", adapter)

# def get_(url):
#     response = session.get(url.strip())
#     return response.json()
# # GET COORDINATES FROM CITY NAME

# def get_place_osmid(county,city, country):
#     url  =  'https://nominatim.openstreetmap.org/search?'
#     # url += f'q={adress}%2C+'
#     url += f'city={city}&'
#     url += f'country={country}&'
#     url += f'county={county}&'


#     url +=  'format=geojson'#&polygon_geojson=1'
#     return get_(url)


# def get_osm_infos(gdf):
#     total = gdf.shape[0]
#     for i, (n, row) in enumerate(gdf.iterrows()):
#         print(f'{i/total:.2%}                   ', end='\r')

#         data = get_place_osmid('', row['libgeo'], 'France')

#         try:
#             gdf.loc[n, 'osm_id'] = data['features'][0]['properties']['osm_id']
#         except:
#             gdf.loc[n, 'osm_id'] = None
#         try:
#             gdf.loc[n, 'place_id'] = data['features'][0]['properties']['place_id']
#         except:
#             gdf.loc[n, 'place_id'] = None
#         try:
#             gdf.loc[n, 'place_rank'] = data['features'][0]['properties']['place_rank']
#         except:
#             gdf.loc[n, 'place_rank'] = None
#         try:
#             gdf.loc[n, 'display_name'] = data['features'][0]['properties']['display_name']
#         except:
#             gdf.loc[n, 'display_name'] = None
#         try:
#             gdf.loc[n, 'importance'] = data['features'][0]['properties']['importance']
#         except:
#             gdf.loc[n, 'importance'] = None
            
#     return gdf


# try: 
#     print('Loading data from local file...')
#     decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
    
# except:
#     print('Loading data from url...')

#     s = requests.get(url).content
#     decoupage = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
#     print('Saving data to local file...')
#     decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

# # Check if osm_id is present and complete gdf if needed
# if 'osm_id' not in decoupage.columns:
#     print('Adding osm feature (approx 5 hours to complete!!!)')
#     decoupage = get_osm_infos(decoupage)
#     print('Saving data to local file...')
#     decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

# display(decoupage.info())
# display(decoupage.head())

In [ ]:
data_path = './datas'
file_name = 'decoupage_administratif'
decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
decoupage.plot()

In [ ]:
# import requests
# from requests.adapters import HTTPAdapter
# import urllib3
# from urllib3 import Retry

# session = requests.Session()
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
# session.headers.update(headers)
# adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
# session.mount("http://", adapter)
# session.mount("https://", adapter)

# def get_(url):
#     response = session.get(url.strip())
#     return response.json()
# # GET COORDINATES FROM CITY NAME

# def get_place_osmid(county,city, country):
#     url  =  'https://nominatim.openstreetmap.org/search?'
#     # url += f'q={adress}%2C+'
#     url += f'city={city}&'
#     url += f'country={country}&'
#     url += f'county={county}&'


#     url +=  'format=geojson'#&polygon_geojson=1'
#     return get_(url)


In [ ]:
# import concurrent.futures



# def get_osm_infos(gdf):
#     total = gdf.shape[0]
#     for i, (n, row) in enumerate(gdf.iterrows()):
#         print(f'{i/total:.2%}                   ', end='\r')
#         data = get_place_osmid('', row['libgeo'], 'France')
#         gdf.loc[n, 'osm_id'] = data['features'][0]['properties']['osm_id']
#         gdf.loc[n, 'place_id'] = data['features'][0]['properties']['place_id']
#         gdf.loc[n, 'place_rank'] = data['features'][0]['properties']['place_rank']
#         gdf.loc[n, 'display_name'] = data['features'][0]['properties']['display_name']
#         gdf.loc[n, 'importance'] = data['features'][0]['properties']['importance']
#     return gdf


# def multi_process_osmid(gdf, processes=4):
#     # Split the DataFrame into four equal chunks
#     num_chunks = processes
#     chunk_size = len(gdf) // num_chunks
#     gdf_chunks = [gdf[i:i+chunk_size].copy() for i in range(0, len(gdf), chunk_size)]

#     # Create a ThreadPoolExecutor to process chunks asynchronously
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Submit the processing function for each chunk
#         futures = [executor.submit(get_osm_infos, chunk) for chunk in gdf_chunks]

#         # Wait for all futures to complete
#         concurrent.futures.wait(futures)

#         # Get the results from the completed futures
#         results = pd.concat([future.result() for future in futures] )
#     return results       


In [ ]:
# decoupage = get_osm_infos(decoupage)

In [ ]:
# decoupage = multi_process_osmid(decoupage, processes=2)